# Functions

> Some commonly used SQL Functions.

In [ ]:
#| default_exp functions
#| hide
from nbdev.showdoc import *
from fastcore.test import *
# allow multiple output from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2

In [ ]:
#|export
from pikaQ.utils import execute
from pikaQ.terms import FieldBase, Field, custom_func

In [ ]:
#|export
class CustomFunction(FieldBase):
    def __init__(self, func_name, arg_names) -> None:
        super().__init__()
        self.func_name = func_name
        self.arg_names = arg_names
        self.get_sql = self.execute
    
    def __call__(self, *args):
        if len(args) != len(self.arg_names):
            raise ValueError(f"The number of args provided {len(args)} is not the same as the number of args expected by this function ({len(self.arg_names)})!")
        def func(*args):
            return f"{self.func_name}({', '.join(args)})"

        self.func = func
        self.args = args
        return self

    def execute(self, **kwargs):
        args = [execute(arg, **kwargs) for arg in self.args]
        return self.func(*args)



`CustomFunction` is a convenient class to create a custom SQL function with the name, and positional arguments, if you don't need the function to be parsed differently for different dialects.

In [ ]:
date_diff = CustomFunction('DATE_DIFF', ['interval', 'start_date', 'end_date'])
date_diff('day', 'start_date', 'end_date').get_sql()

'DATE_DIFF(day, start_date, end_date)'

In [ ]:
date_diff = CustomFunction('DATE_DIFF', ['interval', 'start_date', 'end_date'])
test_eq(date_diff('month', Field('date1'), Field('date2')).get_sql(), 'DATE_DIFF(month, date1, date2)')